## Conversion from convolution filter to binary filter

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
assert tf.__version__ >= "2.7.0"

In [3]:
tf.config.run_functions_eagerly(True)
tf.executing_eagerly()
tf.data.experimental.enable_debug_mode()

In [4]:
# Model / data parameters
num_classes = 20
input_shape = (32, 32, 3)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode='coarse')

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to one-hot representations
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

169017344/169001437 [==============================] - 3s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [15]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

model_AlexNet_full_precision = keras.Sequential(
    [
        keras.Input(shape=input_shape),


        layers.Conv2D(128, kernel_size=(5, 5), activation="relu", padding = "same"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(256, kernel_size=(3, 3), activation="relu", padding = "same"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        layers.Conv2D(512, kernel_size=(3, 3), activation="relu", padding = "same"),
        layers.Conv2D(256, kernel_size=(3, 3), activation="relu", padding = "same"),
        layers.Conv2D(256, kernel_size=(3, 3), activation="relu", padding = "same"),
        
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        layers.Dropout(0.5),

        layers.Flatten(),
        layers.Dense(1024, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(num_classes, activation="softmax")
    ]
)

model_AlexNet_full_precision.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 32, 32, 128)       9728      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 16, 16, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 16, 16, 256)       295168    
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 8, 8, 256)        0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 8, 8, 512)         1180160   
                                                                 
 conv2d_13 (Conv2D)          (None, 8, 8, 256)        

In [16]:
batch_size = 64
epochs = 10

model_AlexNet_full_precision.compile(loss="categorical_crossentropy", optimizer="adam", metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.TopKCategoricalAccuracy(k=3)],run_eagerly=True)

model_AlexNet_full_precision.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/10
704/704 [==============================] - 129s 181ms/step - loss: 2.6979 - categorical_accuracy: 0.1496 - top_k_categorical_accuracy: 0.3498 - val_loss: 2.5033 - val_categorical_accuracy: 0.2176 - val_top_k_categorical_accuracy: 0.4462
Epoch 2/10
704/704 [==============================] - 126s 179ms/step - loss: 2.3423 - categorical_accuracy: 0.2698 - top_k_categorical_accuracy: 0.5120 - val_loss: 2.2536 - val_categorical_accuracy: 0.3088 - val_top_k_categorical_accuracy: 0.5394
Epoch 3/10
704/704 [==============================] - 125s 178ms/step - loss: 2.1694 - categorical_accuracy: 0.3224 - top_k_categorical_accuracy: 0.5751 - val_loss: 2.1084 - val_categorical_accuracy: 0.3482 - val_top_k_categorical_accuracy: 0.5906
Epoch 4/10
704/704 [==============================] - 125s 178ms/step - loss: 2.0250 - categorical_accuracy: 0.3662 - top_k_categorical_accuracy: 0.6234 - val_loss: 2.0345 - val_categorical_accuracy: 0.3734 - val_top_k_categorical_accuracy: 0.6196
Epoch 5/

In [17]:
import time

start_time = time.time()

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model_AlexNet_full_precision.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

print("Time taken for prediction %s" % (time.time() - start_time))

Evaluate on test data
79/79 [==============================] - 7s 83ms/step - loss: 1.7491 - categorical_accuracy: 0.4669 - top_k_categorical_accuracy: 0.7045
test loss, test acc: [1.7491263151168823, 0.4668999910354614, 0.7045000195503235]
Time taken for prediction 7.195261716842651
